<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/algo_min_max_tic_tac_toe/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_minmax_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [ ]:
from collections import defaultdict
import random
import math

# ----------------------------- Game utilities --------------------------------

# Board indices: 0 1

#                2 3

WIN_LINES = [(0,1), (2,3), (0,2), (1,3), (0,3), (1,2)]  # all 2-in-line possibilities

EMPTY = 0
MAX = 1   # X
MIN = -1  # O

def is_terminal(board):
    """Determines if the given board state is terminal and returns the reward.

    Args:
        board (tuple): A tuple representing the current state of the game board.

    Returns:
        tuple: A tuple containing:
            - terminal (bool): True if the board is a terminal state (win, loss, or draw), False otherwise.
            - reward (int): +1 if MAX wins, -1 if MIN wins, 0 for a draw or non-terminal state.
    """
    # check wins
    for (i,j) in WIN_LINES:
        if board[i] == board[j] != EMPTY:
            return True, (1 if board[i] == MAX else -1)
    # draw (all filled)
    if all(cell != EMPTY for cell in board):
        return True, 0
    return False, 0

def legal_actions(board):
    """Returns a list of legal actions (empty cell indices) for the current board state.

    Args:
        board (tuple): A tuple representing the current state of the game board.

    Returns:
        list: A list of integers, where each integer is the index of an empty cell.
    """
    return [i for i,v in enumerate(board) if v == EMPTY]

def apply_action(board, action, player):
    """Applies a given action to the board for a specified player.

    Args:
        board (tuple): The current state of the game board.
        action (int): The index of the cell where the player wants to make a move.
        player (int): The player making the move (MAX=1 or MIN=-1).

    Returns:
        tuple: A new tuple representing the board state after the action has been applied.
    """
    new = list(board)
    new[action] = player
    return tuple(new)

# -------------------------- Truth-table mapping --------------------------------

def state_propositions(board):
    """Returns a dictionary of boolean propositions for a given board state.

    Example propositions: cell_i_is_X, cell_i_is_O for i in 0..3,
    X_two_in_line_threat, O_two_in_line_threat, any_center_empty.

    Args:
        board (tuple): A tuple representing the current state of the game board.

    Returns:
        dict: A dictionary where keys are proposition names (strings) and values are booleans.
    """
    props = {}
    for i in range(4):
        props[f'cell_{i}is_X'] = (board[i] == MAX)
        props[f'cell{i}_is_O'] = (board[i] == MIN)

    def has_threat(player):
        """Helper function to check for two-in-line threats."""
        for (a,b) in WIN_LINES:
            # X_has_threat: True if the player has one piece on a winning line and the other cell is empty.
            if (board[a] == player and board[b] == EMPTY) or (board[b] == player and board[a] == EMPTY):
                return True
        return False

    # X_has_threat: Proposition indicating if MAX (X) has a potential winning line with one piece and one empty cell.
    props['X_has_threat'] = has_threat(MAX)
    # O_has_threat: Proposition indicating if MIN (O) has a potential winning line with one piece and one empty cell.
    props['O_has_threat'] = has_threat(MIN)

    # A simple 'center' concept for the tiny board: cells 1 and 2 are considered 'center-ish'
    # center_any_empty: Proposition indicating if any of the 'center' cells (1 or 2) are empty.
    props['center_any_empty'] = (board[1] == EMPTY or board[2] == EMPTY)

    return props

def propositions_to_key(props):
    """Creates a deterministic string key from a dictionary of propositions.
    This key represents a unique 'truth-table row' for the state.

    Args:
        props (dict): A dictionary of boolean propositions.

    Returns:
        str: A string formed by concatenating '1' for True and '0' for False
             values of propositions, sorted by key name for canonical order.
    """
    keys = sorted(props.keys())
    bits = ['1' if props[k] else '0' for k in keys]
    return ''.join(bits)

# ---------------------------- Minimax search ----------------------------------

def minimax_value(board, player):
    """Calculates the minimax value for a given board state for the specified player.

    This function recursively explores the game tree to determine the optimal
    outcome for the 'player' assuming both players play optimally.

    Args:
        board (tuple): The current state of the game board.
        player (int): The current player whose turn it is (MAX=1 or MIN=-1).

    Returns:
        int: The optimal value of the state from the current player's perspective
             (+1 for win, -1 for loss, 0 for draw).
    """
    term, reward = is_terminal(board)
    if term:
        return reward

    if player == MAX:
        best = -math.inf
        for a in legal_actions(board):
            val = minimax_value(apply_action(board,a,MAX), MIN)
            if val > best:
                best = val
        return best
    else:
        best = math.inf
        for a in legal_actions(board):
            val = minimax_value(apply_action(board,a,MIN), MAX)
            if val < best:
                best = val
        return best

def minimax_policy(board, player):
    """Determines the optimal actions for a player using the minimax algorithm.

    This function finds all legal moves that lead to the best possible outcome
    for the current player, assuming optimal play from both sides.

    Args:
        board (tuple): The current state of the game board.
        player (int): The player for whom to determine the optimal actions (MAX=1 or MIN=-1).

    Returns:
        list: A list of integers, where each integer is an index representing an optimal action.
              Returns an empty list if the board is a terminal state.
    """
    term, _ = is_terminal(board)
    if term:
        return []
    best_actions = []
    best_val = -math.inf if player == MAX else math.inf
    for a in legal_actions(board):
        val = minimax_value(apply_action(board,a,player), -player)
        if player == MAX:
            if val > best_val:
                best_val = val
                best_actions = [a]
            elif val == best_val:
                best_actions.append(a)
        else:
            if val < best_val:
                best_val = val
                best_actions = [a]
            elif val == best_val:
                best_actions.append(a)
    return best_actions

# -------------------------- Tabular Q-learning ---------------------------------

class QLearner:
    """Implements a Q-learning agent for learning optimal policies in a game.

    The Q-learner uses a tabular approach to store Q-values for state-action pairs
    and updates them based on rewards received and future expected values.
    """
    def __init__(self, alpha=0.5, gamma=0.99, epsilon=0.2):
        """Initializes the QLearner with learning parameters.

        Args:
            alpha (float): The learning rate, controlling how much new information overrides old information.
            gamma (float): The discount factor, determining the importance of future rewards.
            epsilon (float): The exploration-exploitation trade-off parameter.
                             (Probability of choosing a random action).
        """
        # Q[state_key][action] = value
        self.Q = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def get_Q(self, state_key, action):
        """Retrieves the Q-value for a given state-action pair.

        Args:
            state_key (str): The unique string representation of the state.
            action (int): The action taken from that state.

        Returns:
            float: The Q-value associated with the state-action pair.
        """
        return self.Q[state_key][action]

    def choose_action(self, state_key, legal_actions_list):
        """Selects an action using an epsilon-greedy policy.

        Args:
            state_key (str): The unique string representation of the current state.
            legal_actions_list (list): A list of legal actions available in the current state.

        Returns:
            int: The chosen action (an index).
        """
        # epsilon-greedy: with probability epsilon, choose a random action.
        if random.random() < self.epsilon:
            return random.choice(legal_actions_list)
        # Otherwise, pick the action with the maximum Q-value.
        qs = [(self.get_Q(state_key,a),a) for a in legal_actions_list]
        maxq = max(qs, key=lambda x: x[0])[0]
        best = [a for q,a in qs if q==maxq]
        return random.choice(best)

    def update(self, s_key, a, r, s_next_key, legal_a_next, opponent_policy):
        """Updates the Q-value for a state-action pair using the Q-learning update rule.

        Args:
            s_key (str): The key for the previous state (current state where action 'a' was taken).
            a (int): The action taken from state `s_key`.
            r (int): The immediate reward received after taking action 'a'.
            s_next_key (str or None): The key for the next state. None if `s_next_key` is a terminal state.
            legal_a_next (list): A list of legal actions available in the next state `s_next_key`.
            opponent_policy (function): The opponent's policy function (e.g., minimax_policy).
        """
        # Since opponent is fixed, the next state's value is expectation under opponent policy
        # target = r + gamma * max_a' E_{opponent}[ Q(s', a') ]
        if not legal_a_next:  # next is terminal
            target = r
        else:
            # compute expected Q for each candidate next action a' (MAX's action)
            best_values = []
            for a_prime in legal_a_next:
                # opponent will respond according to their policy; here we look up Q(s',a')
                # for MAX's potential actions in the next state.
                best_values.append(self.get_Q(s_next_key, a_prime))
            # The Q-learner (MAX) aims to maximize its expected future reward, so it considers the max Q-value.
            target = r + self.gamma * max(best_values)
        # TD update: Q(s,a) = Q(s,a) + alpha * (target - Q(s,a))
        cur = self.get_Q(s_key, a)
        self.Q[s_key][a] = cur + self.alpha * (target - cur)

# ------------------------ Environment & Training loop --------------------------

def play_episode(qlearner, opponent_policy_func, train=True, verbose=False):
    """Simulates a single episode of the game between the Q-learner (MAX) and an opponent (MIN).

    Args:
        qlearner (QLearner): The Q-learning agent for player MAX.
        opponent_policy_func (function): A function representing the opponent's policy (e.g., minimax_policy).
        train (bool): If True, the Q-learner updates its Q-values during the episode.
        verbose (bool): If True, prints game progress.

    Returns:
        int: The final reward for MAX (+1 for win, -1 for loss, 0 for draw).
    """
    board = (0,0,0,0)
    player = MAX
    history = []

    while True:
        term, reward = is_terminal(board)
        if term:
            # If the game is terminal, return the final reward.
            if train:
                # Update for the last transition if necessary (e.g., if MAX made the move leading to terminal state)
                # For this specific implementation, the update happens when MIN makes a move, so nothing extra here.
                pass
            if verbose:
                print('Terminal:', board, 'reward', reward)
            return reward

        if player == MAX:
            # MAX's turn: choose action using the Q-learner's policy (epsilon-greedy).
            props = state_propositions(board)
            s_key = propositions_to_key(props)
            acts = legal_actions(board)
            a = qlearner.choose_action(s_key, acts)
            # Apply MAX's chosen action.
            board_next = apply_action(board, a, MAX)
            # Store MAX's move for potential future Q-value update.
            history.append(('MAX', board, s_key, a))
            board = board_next
            player = MIN

        else:
            # MIN's turn: opponent plays according to its fixed policy (minimax).
            best_actions = opponent_policy_func(board, MIN)
            if not best_actions:
                # This case handles a rare scenario where MIN has no legal moves (e.g., immediate draw/terminal before MIN's turn logic finishes).
                player = MAX
                continue
            a_op = random.choice(best_actions)
            board_next = apply_action(board, a_op, MIN)

            # Q-learner update: If MAX just made a move in the previous step, update its Q-value.
            if history and history[-1][0] == 'MAX':
                _, board_prev, s_key_prev, a_prev = history[-1]
                term_now, reward_now = is_terminal(board_next)
                # Determine the next state key and legal actions for MAX's perspective for the update.
                if term_now:
                    s_next_key = None
                    legal_next = []
                else:
                    props_next = state_propositions(board_next)
                    s_next_key = propositions_to_key(props_next)
                    legal_next = legal_actions(board_next)
                if train:
                    # Perform the Q-value update based on MAX's previous action and the outcome after MIN's response.
                    qlearner.update(s_key_prev, a_prev, reward_now, s_next_key, legal_next, opponent_policy_func)
                # Clear history for this MAX move as it has been processed.
                history.pop()

            board = board_next
            player = MAX

# ----------------------------- Evaluation -------------------------------------

def evaluate(qlearner, opponent_policy_func, episodes=200):
    """Evaluates the performance of the Q-learner against an opponent over multiple episodes.

    Args:
        qlearner (QLearner): The Q-learning agent to evaluate.
        opponent_policy_func (function): The opponent's policy function (e.g., minimax_policy).
        episodes (int): The number of episodes to run for evaluation.

    Returns:
        tuple: A tuple containing:
            - wins (int): The number of episodes won by the Q-learner (MAX).
            - ties (int): The number of episodes that ended in a draw.
            - losses (int): The number of episodes lost by the Q-learner (MAX).
    """
    wins = 0
    ties = 0
    losses = 0
    old_eps = qlearner.epsilon
    qlearner.epsilon = 0.0  # Set epsilon to 0 for greedy evaluation
    for _ in range(episodes):
        res = play_episode(qlearner, opponent_policy_func, train=False, verbose=False)
        if res == 1:
            wins += 1
        elif res == 0:
            ties += 1
        else:
            losses += 1
    qlearner.epsilon = old_eps # Restore original epsilon
    return wins, ties, losses

# ------------------------------- Main -----------------------------------------

if __name__ == '__main__':
    random.seed(42)

    q = QLearner(alpha=0.7, gamma=0.95, epsilon=0.2)

    episodes = 3000
    eval_every = 300

    # Prints an introductory message about the training process.
    print('Training Q-learner against fixed minimax opponent (MIN plays optimal).')
    # Explains the state representation used by the Q-learner.
    print('State representation is a truth-table of propositions (see state_propositions()).')

    for ep in range(1, episodes+1):
        play_episode(q, minimax_policy, train=True)

        if ep % eval_every == 0:
            w,t,l = evaluate(q, minimax_policy, episodes=500)
            # Reports the Q-learner's performance (Wins/Ties/Losses) against the minimax opponent
            # at regular evaluation intervals during training.
            print(f'Episode {ep}: Win/Tie/Loss = {w}/{t}/{l}')

    # Prints a newline for better readability before the final evaluation.
    print('\nFinal evaluation against minimax opponent:')
    w,t,l = evaluate(q, minimax_policy, episodes=2000)
    # Displays the final Win/Tie/Loss record of the Q-learner after all training episodes
    # against the fixed minimax opponent over a larger number of evaluation episodes.
    print('Wins/Ties/Losses:', w, t, l)

    # Inspect learned Q for initial empty board
    init_props = state_propositions((0,0,0,0))
    init_key = propositions_to_key(init_props)
    # Prints the canonical key representation for the initial empty board state based on its propositions.
    print('\nInitial state propositions key:', init_key)
    # Indicates that the following output will list the Q-values for possible actions from the initial state.
    print('Q-values for initial state:')
    for a in legal_actions((0,0,0,0)):
        # Shows the learned Q-value for each legal action from the initial empty board state.
        print('action', a, 'Q=', q.get_Q(init_key,a))

    # Example: show propositions mapping for a sample board
    sample = (1,0,-1,0)
    # Prints a sample board configuration to demonstrate the state representation.
    print('\nSample board', sample)
    # Displays the boolean propositions generated for the `sample` board, illustrating
    # how a board state is translated into features for the Q-learner.
    print('Propositions:', state_propositions(sample))

Training Q-learner against fixed minimax opponent (MIN plays optimal).
State representation is a truth-table of propositions (see state_propositions()).
Episode 300: Win/Tie/Loss = 500/0/0
Episode 600: Win/Tie/Loss = 500/0/0
Episode 900: Win/Tie/Loss = 500/0/0
Episode 1200: Win/Tie/Loss = 500/0/0
Episode 1500: Win/Tie/Loss = 500/0/0
Episode 1800: Win/Tie/Loss = 500/0/0
Episode 2100: Win/Tie/Loss = 500/0/0
Episode 2400: Win/Tie/Loss = 500/0/0
Episode 2700: Win/Tie/Loss = 500/0/0
Episode 3000: Win/Tie/Loss = 500/0/0

Final evaluation against minimax opponent:
Wins/Ties/Losses: 2000 0 0

Initial state propositions key: 00000000001
Q-values for initial state:
action 0 Q= 0.0
action 1 Q= 0.0
action 2 Q= 0.0
action 3 Q= 0.0

Sample board (1, 0, -1, 0)
Propositions: {'cell_0is_X': True, 'cell0_is_O': False, 'cell_1is_X': False, 'cell1_is_O': False, 'cell_2is_X': False, 'cell2_is_O': True, 'cell_3is_X': False, 'cell3_is_O': False, 'X_has_threat': True, 'O_has_threat': True, 'center_any_empty':

# Task
Add comments to the main execution block (if __name__ == '__main__':) in the provided code that explain what each print statement in the output signifies, line by line, to provide a clear understanding of the training and evaluation process.

## Explain code output

### Subtask:
Add comments to the main execution block (if __name__ == '__main__':) that explain what each print statement in the output signifies, line by line, to provide a clear understanding of the training and evaluation process.


## Summary:

### Data Analysis Key Findings
- The main execution block (`if __name__ == '__main__':`) of the code has been thoroughly commented.
- Each print statement in the training and evaluation process output now includes a line-by-line explanation clarifying its significance.
- The comments elucidate the sequential steps, from model initialization through epoch-specific training progress and loss reporting, to the final evaluation metrics.

### Insights or Next Steps
- The added comments provide a clear and explicit understanding of the model's operational flow and performance reporting, enhancing code readability and maintainability.
- This detailed explanation is beneficial for anyone reviewing the code output, allowing for quick comprehension of the training and evaluation phases without needing deep code inspection.
